# Jupyter notebook 05: this notebook use the **Mapillary API** to request metadata only for image candidates selected


## Install necessary libraries

In [ ]:
# Install necessary libraries
%pip install fiona -q

## Import libraries and modules

In [ ]:
# Import library and some pre-installed modules for the pipeline
import os, sys, json, time, gc, math, glob, logging
from pathlib import Path
import numpy as np
import geopandas as gpd
import pandas as pd
import fiona
import torch
import tensorflow as tf
import requests
from time import sleep
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from PIL import Image
from io import BytesIO
from datetime import datetime
# from tqdm import tqdm # ideal para scripts .py
from tqdm.notebook import tqdm # Se estiver usando tqdm no notebook:
from sklearn.neighbors import BallTree
from shapely.geometry import Point, box
from geopy.distance import geodesic
# Importando o módulo de visualização do IPython
from IPython.display import display,Markdown, Image as IPImage

In [2]:
# Sets the root directory of the project as the working directory
os.chdir('..')

In [3]:
# Get current working directory
os.getcwd()

'c:\\DEV\\PhD_Thesis_Step3_OSM_Toponyms'

In [4]:
# Import and Reload the modules to ensure any changes are reflected
import importlib
import src.mapillary_metadata_enricher as mapillary_metadata_enricher

importlib.reload(mapillary_metadata_enricher)

<module 'src.mapillary_metadata_enricher' from 'c:\\DEV\\PhD_Thesis_Step3_OSM_Toponyms\\src\\mapillary_metadata_enricher.py'>

In [5]:
# Função para ler o token do Mapillary
from src.mapillary_metadata_enricher import (
    ler_token_mapillary,
)

TOKEN = ler_token_mapillary()

## Setup GPU

In [6]:
# Configurar para utilizar a GPU T4, caso disponível
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Restringir TensorFlow para utilizar a primeira GPU T4
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Utilizando GPU:", gpus[0])
    except RuntimeError as e:
        # Erro na configuração da GPU
        print(e)
else:
    print("Nenhuma GPU disponível, utilizando CPU.")

Utilizando GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [7]:
# This cell confirm whether TensorFlow is utilizing a GPU and to see which specific GPU is being used
device_name = tf.test.gpu_device_name()

if device_name:
    print(f'A GPU ativa é: {device_name}')
else:
    print('Nenhuma GPU está ativa.')

A GPU ativa é: /device:GPU:0


In [8]:
# The nvidia-smi command-line allow verify that the correct GPU is being used and for monitoring
# its performance and resource usage
!nvidia-smi

Mon Aug 25 10:41:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  |   00000000:01:00.0  On |                  Off |
|  0%   36C    P2             55W /  450W |    1148MiB /  24564MiB |      3%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Load input data
 - Points of interest (POIs) from OpenStreetMap (OSM) with toponyms to validate.
 - Points from Mapillary Coverage Tiles with metadata.

In [9]:
# POIs with toponyms from OSM to validate
# Caminho do arquivo
osm_path = "results/2_toponyms_retrieval/step7_latest_name_ohsome"
osm_fp = os.path.join(osm_path, "step7_consolidado_ohsome_filtrado.gpkg")

# Leitura única
gdf_osm = gpd.read_file(osm_fp)
print(f"Arquivo carregado: {len(gdf_osm)} features.")

# iteração com tqdm
for _, row in tqdm(gdf_osm.iterrows(), total=len(gdf_osm), desc="Iterando pelos POIs do OSM"):
    # Seu processamento aqui
    pass

Arquivo carregado: 1627 features.


Iterando pelos POIs do OSM:   0%|          | 0/1627 [00:00<?, ?it/s]

In [ ]:
display(gdf_osm.head())

In [11]:
len(gdf_osm)

1627

In [ ]:
# column names of the "gdf_osm" GeoDataFrame
gdf_osm.columns.tolist()

In [ ]:
# get CRS gdf_osm
gdf_osm.crs

In [ ]:
# Points from Mapillary coverage tiles (carga otimizada de dados)
import gc

mapillary_fp = "results/3_mapillary_coverage/20250620_mpl_coverage_bh.gpkg"

# Conte o total de features
with fiona.open(mapillary_fp) as src:
    total_features = len(src)
print(f"Total de feições: {total_features}")

chunk_size = 5000
dfs = []

with fiona.open(mapillary_fp) as src:
    features = []
    # Barra de progresso!
    for i, feat in enumerate(tqdm(src, total=total_features, desc="Carregando pontos")):
        features.append(feat)
        if (i+1) % chunk_size == 0 or (i+1) == total_features:
            df_chunk = gpd.GeoDataFrame.from_features(features, crs=src.crs)
            dfs.append(df_chunk)
            # Limpa variáveis e força garbage collector
            del df_chunk
            features = []
            gc.collect()

# Concatenar todos os chunks em um único GeoDataFrame
gdf_mapillary = gpd.GeoDataFrame(pd.concat(dfs, ignore_index=True), crs=dfs[0].crs)
print("GeoDataFrame final:", gdf_mapillary.shape)

# Liberar a memória dos dfs intermediários
del dfs
gc.collect()

In [ ]:
display(gdf_mapillary.head())

In [ ]:
# size of image points from Mapillary coverage tiles
len(gdf_mapillary)

In [ ]:
# lista todas colunas gdf_mapillary
gdf_mapillary.columns.tolist()

In [ ]:
# Get CRS gdf_mapillary
gdf_mapillary.crs

In [ ]:
# Check data types
print("Data types of gdf_osm:")
print(gdf_osm.dtypes)

print("\nData types of gdf_mapillary:")
print(gdf_mapillary.dtypes)

In [ ]:
# Visualizar no folium com tooltips os GeoDataFrames
import folium

# --- Funções para tooltips customizados ---

def make_osm_tooltip(row):
    campos = ['id_celula', 'classe', 'tag', 'value', '@osmId', '@timestamp', '@version', 'name']
    label = ""
    for k in campos:
        if k in row and pd.notnull(row[k]):
            if k == "classe" or k == "name":
                label += f"<b>{k}:</b> <b>{row[k]}</b><br>"
            else:
                label += f"<b>{k}:</b> {row[k]}<br>"
    return label

def make_mapillary_tooltip(row):
    campos = ['image_id', 'sequence_id',
              'thumb_256_url', 'thumb_1024_url', 'thumb_2048_url', 'thumb_original_url',
              'captured_date']
    label = ""
    for k in campos:
        if k in row and pd.notnull(row[k]):
            # Coloca thumbs como links clicáveis
            if k.startswith("thumb_"):
                url = row[k]
                label += f'<b>{k}:</b> <a href="{url}" target="_blank">Abrir imagem</a><br>'
            else:
                label += f"<b>{k}:</b> {row[k]}<br>"
    return label

# --- Centralizar o mapa ---
def get_center(gdf):
    bounds = gdf.total_bounds
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2
    return [center_lat, center_lon]

m = folium.Map(location=get_center(gdf_osm), zoom_start=14, tiles='OpenStreetMap')

# --- OSM: marcador default de localização ---
if not gdf_osm.empty:
    for _, row in gdf_osm.iterrows():
        geom = row.geometry
        if geom.geom_type == 'Point':
            folium.Marker(
                location=[geom.y, geom.x],
                icon=folium.Icon(icon="glyphicon glyphicon-map-marker", prefix='glyphicon', color='blue', icon_color='white'),
                popup=folium.Popup(make_osm_tooltip(row), max_width=350)
            ).add_to(m)

# --- Mapillary: círculo pequeno com tooltip com thumbs clicáveis ---
if not gdf_mapillary.empty:
    for _, row in gdf_mapillary.iterrows():
        geom = row.geometry
        if geom.geom_type == 'Point':
            folium.CircleMarker(
                location=[geom.y, geom.x],
                radius=4,
                color='red',
                fill=True,
                fill_color='red',
                fill_opacity=0.7,
                popup=folium.Popup(make_mapillary_tooltip(row),  max_width=350)
            ).add_to(m)

display(m)

## Selection of candidate Mapillary images

**Estratégia Seleção das imagens candidatas**

1. **Pré-processamento e organização espacial**

   - Extrair as coordenadas dos pontos Mapillary no formato (latitude, longitude).
   - Converter essas coordenadas para radianos, preparando os dados para uso com a métrica Haversine.
   - Construir um índice espacial com a estrutura `BallTree`
     - Esse índece permite consultas eficientes dos K vizinhos mais próximos com base em distância geodésica real (ideal para dados em coordenadas geográficas).

2. **Busca eficiente dos vizinhos mais próximos**

   - Para cada ponto OSM:
     - Consultar o índice `BallTree` para retornar os N (ex: 200) pontos Mapillary mais próximos com base em distância geodésica (em metros).
     - Registrar a distância de cada vizinho ao ponto OSM para fins de ranqueamento, visualização e controle de qualidade.
     - Encaminhar os vizinhos encontrados para a etapa de avaliação visual com CLIP.

In [ ]:
# Coletar image_ids candidatos (k_vizinhos)
k_vizinhos = 200  # Número de vizinhos mais próximos a serem consultados

# Extrai coordenadas do Mapillary (lat, lon) em radianos
gdf_mapillary["coords"] = gdf_mapillary.geometry.apply(lambda g: (g.y, g.x))
coords_mapillary_rad = np.radians(gdf_mapillary["coords"].to_list())

# Cria índice BallTree com métrica haversine
tree = BallTree(coords_mapillary_rad, metric='haversine')

# Coletar todos os image_ids dos k_vizinhos mais próximos de cada ponto OSM
image_ids_candidatos = set()

for idx, row in tqdm(gdf_osm.iterrows(), total=len(gdf_osm), desc="Coletando image_ids candidatos"):
    lat_osm, lon_osm = row.geometry.y, row.geometry.x
    coord_osm_rad = np.radians([[lat_osm, lon_osm]])
    dist_rad, ind = tree.query(coord_osm_rad, k=k_vizinhos)
    candidatos = gdf_mapillary.iloc[ind[0]]
    image_ids_candidatos.update(candidatos["image_id"].values)

print(f"Total de image_ids únicos candidatos: {len(image_ids_candidatos)}")

In [ ]:
# Salvar os IDs das imagens candidatas em um CSV para retomar futuramente sem recalcular
out_path = "results/3_mapillary_coverage/mapillary_candidatos_subset"
os.makedirs(out_path, exist_ok=True)

# Garantir que os tipos batem: transforme tudo para string
image_ids_candidatos_str = {str(x) for x in image_ids_candidatos}

ids_csv = "mapillary_image_ids_candidatos.csv"
pd.Series(sorted(image_ids_candidatos_str), name="image_id").to_csv(os.path.join(out_path, ids_csv), index=False)
print(f"IDs candidatos salvos em: {ids_csv} (total={len(image_ids_candidatos_str)})")

In [ ]:
# With necessary reload  image_ids_candidatos.csv
out_path = "results/3_mapillary_coverage/mapillary_candidatos_subset"
ids_csv = "mapillary_image_ids_candidatos.csv"

ids_csv_path = os.path.join(out_path, ids_csv)

if os.path.exists(ids_csv_path):
    df_ids = pd.read_csv(ids_csv_path)
    image_ids_candidatos_str = set(df_ids["image_id"].astype(str).tolist())
    image_ids_candidatos = set(df_ids["image_id"].tolist()) # Keep original type if needed
    print(f"IDs candidatos carregados de: {ids_csv_path} (total={len(image_ids_candidatos)})")
else:
    print(f"Arquivo de IDs candidatos não encontrado: {ids_csv_path}")
    # Or raise an error
    raise FileNotFoundError(f"Arquivo de IDs candidatos não encontrado: {ids_csv_path}")

In [ ]:
# Filtrar gdf_mapillary através dos image_ids_candidatos
mask = gdf_mapillary["image_id"].astype(str).isin(image_ids_candidatos_str)
gdf_mapillary_candidatos = gdf_mapillary.loc[mask].copy()

# Remover duplicatas por image_id (pode acontecer por sobreposição entre OSMs)
gdf_mapillary_candidatos = gdf_mapillary_candidatos.drop_duplicates(subset=["image_id"])

print("Subset final:", gdf_mapillary_candidatos.shape)

# Salvar GeoPackage
gpkg_path = os.path.join(out_path, "mapillary_coverage_bh_meta_candidatos.gpkg")
gdf_mapillary_candidatos.to_file(gpkg_path, driver="GPKG", layer="mapillary_coverage_bh_meta_candidatos")
print("GPKG salvo em:", gpkg_path)

In [ ]:
# Reload mapillary_coverage_bh_meta.gpkg (carga otimizada de dados)

import gc

out_path = "results/3_mapillary_coverage/mapillary_candidatos_subset"
fp_mapillary_candidatos = os.path.join(out_path, "mapillary_coverage_bh_meta_candidatos.gpkg")

# Conte o total de features
with fiona.open(fp_mapillary_candidatos) as src:
    total_features = len(src)
print(f"Total de feições: {total_features}")

chunk_size = 10000
dfs = []

with fiona.open(fp_mapillary_candidatos) as src:
    features = []
    # Barra de progresso!
    for i, feat in enumerate(tqdm(src, total=total_features, desc="Carregando pontos")):
        features.append(feat)
        if (i+1) % chunk_size == 0 or (i+1) == total_features:
            df_chunk = gpd.GeoDataFrame.from_features(features, crs=src.crs)
            dfs.append(df_chunk)
            # Limpa variáveis e força garbage collector
            del df_chunk
            features = []
            gc.collect()

# Concatenar todos os chunks em um único GeoDataFrame
gdf_mapillary_candidatos = gpd.GeoDataFrame(pd.concat(dfs, ignore_index=True), crs=dfs[0].crs)
print("GeoDataFrame final:", gdf_mapillary_candidatos.shape)

# Liberar a memória dos dfs intermediários
del dfs
gc.collect()

In [ ]:
display(gdf_mapillary_candidatos.head())

### Retrieve Mapillary metadata only for image candidates selected

In [ ]:
# This code cell enriches the Mapillary points retrived from Mapillary Coverage tiles with image metadata
# To apply this code, you need to have the Mapillary API token set up in your environment and 
# the module mapillary_metadata_enricher should be imported

from src.mapillary_metadata_enricher import (
    ler_token_mapillary,
    enriquecer_geodataframe,
    salvar_geodataframe_enriquecido
)
import geopandas as gpd
import time
from tqdm.notebook import tqdm
import logging

# Configurar logging detalhado
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 1. Configuração inicial
TOKEN = ler_token_mapillary()
INPUT_GPKG = "results/3_mapillary_coverage/mapillary_coverage_bh_tmp3.gpkg"
OUTPUT_GPKG = "results/3_mapillary_coverage/mapillary_coverage_bh_tmp3_meta.gpkg"

# 2. Carregar dados existentes
logger.info(f"Carregando dados de {INPUT_GPKG}")
gdf = gpd.read_file(INPUT_GPKG, layer='mapillary_coverage_bh')
logger.info(f"Total de pontos carregados: {len(gdf)}")
logger.info(f"Colunas originais: {list(gdf.columns)}")

# 3. Enriquecer dados
start_time = time.time()
gdf_enriched = enriquecer_geodataframe(
    gdf,
    TOKEN,
    max_workers=15,
    batch_size=10000
)
enrichment_time = time.time() - start_time

logger.info(f"\nEnriquecimento concluído em {enrichment_time/60:.2f} minutos")
logger.info(f"Colunas originais: {len(gdf.columns)}")
logger.info(f"Colunas enriquecidas: {len(gdf_enriched.columns)}")
logger.info(f"Novas colunas adicionadas: {len(gdf_enriched.columns) - len(gdf.columns)}")

# 4. Salvar resultados
salvar_geodataframe_enriquecido(gdf_enriched, OUTPUT_GPKG)

### Update Mapillary metadata only for image candidates selected

 - Essa implemetação foi utilizada, pois já tinha-se o conjunto inteiro de imagens do mapillary com todos os metadados preenchidos e desejava-se atualizar as thumbs_urls apenas dos candidados candidatos
 
 - O ideial é requisitar todos os metadados das iamgens nesse momento

In [ ]:
# Função para atualizar URLs dos thumbnails apenas para os pontos candidatos
# implementação para salvar parciais das URLs atualizadas a cada N batches com retomada automática a partir do último Batch

from pathlib import Path

def atualizar_urls_mapillary(
    image_ids, token, batch_size=100, sleep_time=0.2,
    salvar_a_cada=10, # Salva a cada N batches
    out_dir="results/3_mapillary_coverage/mapillary_candidatos_subset/mapillary_partial_urls_updated",
    base_name="urls_atualizados"):

    """
    Atualiza as URLs de thumbnails para um conjunto de image_ids do Mapillary.
    Salva parciais a cada N batches e pode retomar de onde parou.
    """

    Path(out_dir).mkdir(parents=True, exist_ok=True)
    ultimo_batch_path = os.path.join(out_dir, f"{base_name}_ultimo_batch.txt")

    # ======= Retomar de onde parou, se arquivo existir =======
    start_i = 0
    atualizados = []
    erros = []

    if os.path.exists(ultimo_batch_path):
        with open(ultimo_batch_path, "r") as f:
            start_i = int(f.read().strip())
        # Carrega parciais já salvos (último batch)
        try:
            df_tmp = pd.read_csv(os.path.join(out_dir, f"{base_name}_PARCIAL_{start_i}.csv"))
            atualizados = df_tmp.to_dict("records")
            print(f"[RETOMANDO] Iniciando a partir do batch {start_i} ({len(atualizados)} já processados)")
        except Exception as e:
            print("[AVISO] Não foi possível carregar o último parcial salvo. Iniciando do zero.")

        try:
            erros_tmp = pd.read_csv(os.path.join(out_dir, f"{base_name}_ERROS_{start_i}.csv"))
            erros = erros_tmp.to_dict("records")
        except Exception as e:
            pass  # Não tem erro salvo, segue

    batch_counter = start_i // batch_size

    # ========== PROCESSAMENTO ===============
    for i in tqdm(range(start_i, len(image_ids), batch_size), desc="Atualizando URLs"):
        batch = list(image_ids)[i:i+batch_size]
        for image_id in batch:
            url = f"https://graph.mapillary.com/{image_id}?fields=thumb_256_url,thumb_1024_url,thumb_2048_url,thumb_original_url"
            headers = {"Authorization": f"OAuth {token}"}
            try:
                response = requests.get(url, headers=headers, timeout=10)
                if response.status_code == 200:
                    data = response.json()
                    data['image_id'] = image_id
                    atualizados.append(data)
                else:
                    erros.append({"image_id": image_id, "status": response.status_code, "msg": response.text})
            except Exception as e:
                erros.append({"image_id": image_id, "status": "EXC", "msg": str(e)})
            time.sleep(sleep_time)  # Para evitar bloqueios na API

        batch_counter += 1
        # === Salvamento parcial a cada N batches ===
        if batch_counter % salvar_a_cada == 0:
            df_tmp = pd.DataFrame(atualizados)
            df_tmp.to_csv(os.path.join(out_dir, f"{base_name}_PARCIAL_{i+batch_size}.csv"), index=False)
            pd.DataFrame(erros).to_csv(os.path.join(out_dir, f"{base_name}_ERROS_{i+batch_size}.csv"), index=False)
            with open(ultimo_batch_path, "w") as f:
                f.write(str(i+batch_size))
            print(f"[SALVO PARCIAL] Até image_id {i+batch_size}: {len(df_tmp)} urls atualizadas, {len(erros)} erros")

    # Salvar final
    df_atualizados = pd.DataFrame(atualizados)
    df_atualizados.to_csv(os.path.join(out_dir, f"{base_name}_FINAL.csv"), index=False)
    pd.DataFrame(erros).to_csv(os.path.join(out_dir, f"{base_name}_ERROS_FINAL.csv"), index=False)
    print(f"[SALVO FINAL] {len(df_atualizados)} urls atualizadas, {len(erros)} erros")

    # Apaga arquivo de checkpoint
    if os.path.exists(ultimo_batch_path):
        os.remove(ultimo_batch_path)

    return df_atualizados, erros

In [ ]:
# Executar a atualização das URLs de thumbnails para os image_ids candidatos
out_dir = "results/3_mapillary_coverage/mapillary_candidatos_subset/mapillary_partial_urls_updated"
Path(out_dir).mkdir(parents=True, exist_ok=True)

df_thumbs_atualizados, erros = atualizar_urls_mapillary(
    image_ids=image_ids_candidatos,
    token=TOKEN,
    batch_size=100,
    sleep_time=0.2,
    salvar_a_cada=10,
    out_dir=out_dir
)

In [ ]:
# Funções para recarregar todos os CSVs com as urls atualizadas e consolidar em um único CSV
import os, glob, sqlite3, gc
from pathlib import Path
from tqdm.notebook import tqdm

# === paths ===
out_dir   = "results/3_mapillary_coverage/mapillary_candidatos_subset/mapillary_partial_urls_updated"
base_name = "urls_atualizados"
Path(out_dir).mkdir(parents=True, exist_ok=True)

# arquivo sqlite temporário para consolidação sem RAM
sqlite_path = os.path.join(out_dir, f"{base_name}_consolidacao.sqlite")

# remove sqlite antigo
if os.path.exists(sqlite_path):
    os.remove(sqlite_path)

conn = sqlite3.connect(sqlite_path)
cur  = conn.cursor()

# === tabelas (PRIMARY KEY em image_id para REPLACE) ===
cur.execute("""
CREATE TABLE urls (
    image_id TEXT PRIMARY KEY,
    thumb_256_url TEXT,
    thumb_1024_url TEXT,
    thumb_2048_url TEXT,
    thumb_original_url TEXT,
    __src TEXT
)
""")

cur.execute("""
CREATE TABLE erros (
    image_id TEXT PRIMARY KEY,
    status TEXT,
    msg TEXT,
    __src TEXT
)
""")
conn.commit()

def _files(pattern):
    return sorted(glob.glob(os.path.join(out_dir, pattern)))

def _ensure_cols(df, cols):
    for c in cols:
        if c not in df.columns:
            df[c] = None
    return df

# =======================
# 1) CONSOLIDAR ATUALIZAÇÕES (PARCIAL_* + FINAL) SEM ESTOURAR RAM
# =======================
upd_files = _files(f"{base_name}_PARCIAL_*.csv")
final_upd = os.path.join(out_dir, f"{base_name}_FINAL.csv")
if os.path.exists(final_upd):
    upd_files.append(final_upd)

if not upd_files:
    raise RuntimeError("Nenhum CSV parcial/final encontrado para consolidar.")

cols_urls = ["image_id","thumb_256_url","thumb_1024_url","thumb_2048_url","thumb_original_url","__src"]

for f in tqdm(upd_files, desc="Consolidando URLs (arquivos)"):
    # ler em chunks para não estourar memória
    try:
        for chunk in pd.read_csv(f, chunksize=50_000):
            chunk["__src"] = os.path.basename(f)
            chunk = _ensure_cols(chunk, cols_urls)
            # normaliza tipos
            chunk["image_id"] = chunk["image_id"].astype(str)

            # monta tuplas p/ INSERT OR REPLACE
            rows = list(zip(
                chunk["image_id"],
                chunk["thumb_256_url"],
                chunk["thumb_1024_url"],
                chunk["thumb_2048_url"],
                chunk["thumb_original_url"],
                chunk["__src"]
            ))
            cur.executemany("""
                INSERT OR REPLACE INTO urls
                (image_id, thumb_256_url, thumb_1024_url, thumb_2048_url, thumb_original_url, __src)
                VALUES (?, ?, ?, ?, ?, ?)
            """, rows)
            conn.commit()

            # libera RAM do chunk
            del chunk, rows
            gc.collect()
    except Exception as e:
        print(f"[AVISO] falha lendo {f}: {e}")

# exporta consolidado para CSV
consol_csv = os.path.join(out_dir, f"{base_name}_CONSOLIDADO.csv")
df_iter = pd.read_sql_query("SELECT * FROM urls", conn)
df_iter.to_csv(consol_csv, index=False)
print(f"[OK] Consolidado URLs: {consol_csv} | linhas: {len(df_iter)}")
del df_iter
gc.collect()

# =======================
# 2) CONSOLIDAR ERROS (ERROS_* + ERROS_FINAL)
# =======================
err_files = _files(f"{base_name}_ERROS_*.csv")
final_err = os.path.join(out_dir, f"{base_name}_ERROS_FINAL.csv")
if os.path.exists(final_err):
    err_files.append(final_err)

if err_files:
    for f in tqdm(err_files, desc="Consolidando ERROS (arquivos)"):
        try:
            for chunk in pd.read_csv(f, chunksize=50_000):
                chunk["__src"] = os.path.basename(f)
                chunk = _ensure_cols(chunk, ["image_id","status","msg","__src"])
                chunk["image_id"] = chunk["image_id"].astype(str)

                rows = list(zip(
                    chunk["image_id"],
                    chunk["status"],
                    chunk["msg"],
                    chunk["__src"]
                ))
                cur.executemany("""
                    INSERT OR REPLACE INTO erros
                    (image_id, status, msg, __src)
                    VALUES (?, ?, ?, ?)
                """, rows)
                conn.commit()

                del chunk, rows
                gc.collect()
        except Exception as e:
            print(f"[AVISO] falha lendo {f}: {e}")

    err_csv = os.path.join(out_dir, f"{base_name}_ERROS_CONSOLIDADO.csv")
    df_err = pd.read_sql_query("SELECT * FROM erros", conn)
    df_err.to_csv(err_csv, index=False)
    print(f"[OK] Erros consolidados: {err_csv} | linhas: {len(df_err)}")
    del df_err
    gc.collect()
else:
    print("[INFO] Nenhum arquivo de ERROS encontrado.")

# =======================
# 3) PENDENTES (se image_ids_candidatos estiver disponível no ambiente)
# =======================
try:
    ids_target = {str(x) for x in image_ids_candidatos}

    # pega ids_ok diretamente do sqlite (evita carregar CSV gigante)
    ids_ok = set(r[0] for r in cur.execute("SELECT image_id FROM urls"))

    # ids_err (pode não existir tabela vazia)
    try:
        ids_err = set(r[0] for r in cur.execute("SELECT image_id FROM erros"))
    except sqlite3.OperationalError:
        ids_err = set()

    ids_missing = sorted(list(ids_target - ids_ok - ids_err))
    print(f"[RESUMO] candidatos: {len(ids_target)} | atualizados OK: {len(ids_ok)} | erros: {len(ids_err)} | pendentes: {len(ids_missing)}")

    pend_csv = os.path.join(out_dir, f"{base_name}_PENDENTES.csv")
    pd.Series(ids_missing, name="image_id").to_csv(pend_csv, index=False)
    print(f"[OK] Pendentes salvos: {pend_csv}")
except NameError:
    print("[AVISO] 'image_ids_candidatos' não está no ambiente; pendências não geradas.")

# encerra sqlite e limpa
cur.close()
conn.close()
if os.path.exists(sqlite_path):
    os.remove(sqlite_path)  # apaga o banco temporário

gc.collect()
print("[DONE] Consolidação finalizada com uso mínimo de RAM.")

In [ ]:
# Atualizar novamente as URLS dos image_ids com erros (pendentes)
# caminho para o CSV de pendentes
pend_csv = "results/3_mapillary_coverage/mapillary_candidatos_subset/mapillary_partial_urls_updated/urls_atualizados_ERROS_CONSOLIDADO.csv"

# Carregar a lista de image_ids pendentes
pendentes_df = pd.read_csv(pend_csv)
image_ids_pendentes = pendentes_df["image_id"].dropna().astype(str).tolist()

print(f"Total de pendentes: {len(image_ids_pendentes)}")

# reprocessar apenas os pendentes
df_thumbs_atualizados_pend, erros_pend = atualizar_urls_mapillary(
    image_ids=image_ids_pendentes,
    token=TOKEN,
    batch_size=100,
    sleep_time=0.2,
    salvar_a_cada=10,
    out_dir="results/3_mapillary_coverage/mapillary_candidatos_subset/mapillary_partial_urls_updated"
)

print(f"URLs atualizadas para pendentes: {len(df_thumbs_atualizados_pend)}")
print(f"Erros nos pendentes: {len(erros_pend)}")

In [ ]:
# Reload dos arquivos finais para enfim "reconstruir o "df_thumbs_atualizados"

out_dir = Path(r"results/3_mapillary_coverage/mapillary_candidatos_subset/mapillary_partial_urls_updated")
base_name = "urls_atualizados"

# Arquivos de entrada
files_to_load = [
    out_dir / f"{base_name}_CONSOLIDADO.csv",
    out_dir / f"{base_name}_FINAL.csv"
]

dfs = []
for path in tqdm(files_to_load, desc="Carregando CSVs"):
    if path.exists():
        try:
            df = pd.read_csv(path)
            print(f"[OK] Carregado: {path.name} | linhas: {len(df)}")
            dfs.append(df)
            gc.collect()
        except Exception as e:
            print(f"[ERRO] Falha ao carregar {path.name}: {e}")
    else:
        print(f"[AVISO] Arquivo não encontrado: {path.name}")

if not dfs:
    raise RuntimeError("Nenhum arquivo encontrado para carregar.")

# Concatena e limpa intermediários da memória
df_thumbs_atualizados = pd.concat(dfs, ignore_index=True)
del dfs
gc.collect()

# Deduplicar por image_id, mantendo a última ocorrência
df_thumbs_atualizados["image_id"] = df_thumbs_atualizados["image_id"].astype(str)
df_thumbs_atualizados = df_thumbs_atualizados.drop_duplicates(
    subset=["image_id"], keep="last"
).reset_index(drop=True)

print(f"[FINAL] df_thumbs_atualizados gerado com {len(df_thumbs_atualizados)} linhas")

In [ ]:
display(df_thumbs_atualizados.head())

In [ ]:
len(df_thumbs_atualizados)

In [ ]:
# Check data types
print("Data types of gdf_mapillary_candidatos:")
print(gdf_mapillary_candidatos.dtypes)

print("\nData types of df_thumbs_atualizados:")
print(df_thumbs_atualizados.dtypes)

In [ ]:
# Merge das urls atualizados armazenadas no "df_thumbs_atualizados" com o "gdf_mapillary_candidatos"

# Converter ambos para string
gdf_mapillary_candidatos["image_id"] = gdf_mapillary_candidatos["image_id"].astype(str)
df_thumbs_atualizados["image_id"] = df_thumbs_atualizados["image_id"].astype(str)

# Juntar as novas URLs ao GeoDataFrame original (mantém todos os atributos do ponto)
gdf_mapillary_candidatos_atualizado = gdf_mapillary_candidatos.merge(
    df_thumbs_atualizados,
    how='left',
    on='image_id',
    suffixes=('', '_nova')
)
# Usa a URL nova se disponível, senão mantém a antiga
for col in ['thumb_256_url', 'thumb_1024_url', 'thumb_2048_url', 'thumb_original_url']:
    gdf_mapillary_candidatos_atualizado[col] = gdf_mapillary_candidatos_atualizado[col + '_nova'].combine_first(gdf_mapillary_candidatos_atualizado[col])

# Remover as colunas *_nova
gdf_mapillary_candidatos_atualizado = gdf_mapillary_candidatos_atualizado[
    [c for c in gdf_mapillary_candidatos_atualizado.columns if not c.endswith('_nova')]
]

# Salva somente os pontos com URLs atualizadas
gdf_mapillary_candidatos_atualizado = gdf_mapillary_candidatos_atualizado[gdf_mapillary_candidatos_atualizado["image_id"].isin(image_ids_candidatos_str)]
display(gdf_mapillary_candidatos_atualizado.head())

In [ ]:
len(gdf_mapillary_candidatos_atualizado)

In [ ]:
# Check data types
# Veja os tipos das colunas
print(gdf_mapillary_candidatos_atualizado.dtypes)

# Possíveis colunas problemáticas para GPKG (tuplas, listas, dicionários, objetos complexos etc.)
for col in gdf_mapillary_candidatos_atualizado.columns:
    if gdf_mapillary_candidatos_atualizado[col].apply(lambda x: isinstance(x, tuple)).any():
        print(f"Coluna '{col}' contém tupla!")

In [ ]:
# Converte qualquer tupla/lista para string (menos a geometry!)
for col in gdf_mapillary_candidatos_atualizado.columns:
    if col != "geometry":
        gdf_mapillary_candidatos_atualizado[col] = gdf_mapillary_candidatos_atualizado[col].apply(
            lambda x: str(x) if isinstance(x, (tuple, list, dict)) else x
        )

In [ ]:
# Salva o GeoDataFrame atualizado com as URLs corrigidas
gdf_mapillary_candidatos_atualizado.to_file(
    os.path.join("results",
                 "3_mapillary_coverage",
                 "mapillary_candidatos_subset",
                 "mapillary_candidatos_atualizados.gpkg"),
                 driver="GPKG", layer="mapillary_candidatos_atualizados"
)